# Analysis on ResMaskNet & SVM FER method using Padova Data

In [1]:
from feat import Detector
import pandas as pd
import os

In [2]:
detector = Detector(
    face_model="img2pose",
    landmark_model="mobilefacenet",
    au_model='xgb',
    emotion_model="resmasknet",
    facepose_model="img2pose",
    device='cuda' #Use GPU Acceleration
)

In [ ]:
# Directory names
video_clips  = 'data/PEDFE_set_clips/'  
video_labels = 'data/ST1_demo.csv'
labels = pd.read_csv(video_labels)

In [14]:
# Loop through each entry in the PEDFE_code column
for PDFE_CODE in labels['PEDFE_code']:
    video_path = os.path.join(video_clips, f'{PDFE_CODE}.avi')
    print(f'Processing video: {video_path}')
    # Detect with py-feat
    result = detector.detect_video(video_path=video_path, skip_frames=10, batch_size=35)
    # Calculate mean
    result = result.extract_mean()
    # Append emotion prediction result
    result_emo = result[['mean_anger','mean_disgust','mean_fear','mean_happiness','mean_sadness','mean_surprise','mean_neutral']]
    result_emo = pd.DataFrame({'PDFE_CODE': PDFE_CODE, **result_emo}) # Insert PDFE_CODE as the first column
    # Save to CSV
    result_emo.to_csv(f"result_{PDFE_CODE}.csv", index=False)
    print(result_emo)
    break

Processing video: data/PEDFE_set_clips/1_dg_1.avi


100%|██████████| 1/1 [00:03<00:00,  3.06s/it]

  PDFE_CODE  mean_anger  mean_disgust  mean_fear  mean_happiness  \
0    1_dg_1    0.015438      0.572601   0.001519        0.002715   

   mean_sadness  mean_surprise  mean_neutral  
0       0.36904       0.000227       0.03846  
